### General Axissymmetric element
Linear tri, linear and quadratic quad elements for axisymmetric analysis.

In [41]:
import copy
import time
from typing import Callable

import numpy as np
import sympy as sp
sp.init_printing()

### The physical properties of the element

In [42]:
ND = 3  # number of degrees of freedom per node
NNODE = 4  # number of nodes
if NNODE == 3:
    NGS = [1, 3]  # number of Gauss points, can be 1, 3
elif NNODE == 4:
    NGS = [1, 2]  # number of Gauss points, can be 1, 2, 3, 4, 5, or 6
elif NNODE == 8:
    NGS = [1, 2]  # number of Gauss points, can be 1, 2, 3, 4, 5, or 6
else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

### The tri element

```
    ^
    |t
  3 |
  | \
  | |\
  | |-\--->s
  |    \
  1-----2
```

### The linear quad element

Node numbering is CCW, starting at ``(-1, -1)``:
```
     ^
     |t
     |
  4--|--3
  |  |  |
  |  |--|--->s
  |     |
  1-----2
```

### The quadratic quad element

Node numbering is CCW, starting at ``(-1, -1)``:
```
      ^
      |t
      |
  4---|7---3
  |   |    |
  8   |----6--->s
  |        |
  1---5----2

      Z
      ^
      |
  4---7---3
  |       |
  8       6  --> R
  |       |
  1---5---2

```

It has 3 degrees of freedom each node:
- u: horizontal displacement
- theta: circumferential displacement
- v: vertical displacement

The DOFs are ordered as follows: u, theta, v.


In [43]:
### The physical properties of the element
E = 200e9  # Young's modulus in Pa
nu = 0.30  # Poisson's ratio

# Define symbolic physical coordinates
r1, z1, th1, r2, z2, th2, r3, z3, th3, r4, z4, th4, r5, z5, th5, r6, z6, th6, r7, z7, th7, r8, z8, th8 = sp.symbols('r1 z1 th1 r2 z2 th2 r3 z3 th3 r4 z4 th4 r5 z5 th5 r6 z6 th6 r7 z7 th7 r8 z8 th8')

# Define symbolic variables. Capital letters R, Z are the symbols for the variables r, z
R, Z, TH = sp.symbols('R Z TH')  # radial [m], vertical [m] and circumferential [°] coordinates

if NNODE == 3:
    # Example coordinates for the nodes
    # physical_coords = {'P1': (0, 0, 0),
    #                    'P2': (1, 0, 0),
    #                    'P3': (0, 1, 0)}  # example coordinates for the nodes
    # physical_coords = {'P1': (3, 0, 0),
    #                    'P2': (4, 0, 0),
    #                    'P3': (3, 0, 1)}  # example coordinates for the nodes

    physical_coords = {'P1': (10000, 0, 0),
                       'P2': (10020, 0, 200),
                       'P3': (10000, 0, 1000)}  # example coordinates for the nodes

    midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0]) / 3, \
                (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1]) / 3, \
        (physical_coords['P1'][2] + physical_coords['P2'][2] + physical_coords['P3'][2]) / 3

    physical_coords_map = {'r1': physical_coords['P1'][0], 'th1': physical_coords['P1'][1], 'z1': physical_coords['P1'][2],
                           'r2': physical_coords['P2'][0], 'th2': physical_coords['P2'][1], 'z2': physical_coords['P2'][2],
                           'r3': physical_coords['P3'][0], 'th3': physical_coords['P3'][1], 'z3': physical_coords['P3'][2],}
    # Define the physical nodes as a matrix so it can be used a symbolic evaluation
    physical_nodes = sp.Matrix([[r1, th1, z1],
                                [r2, th2, z2],
                                [r3, th3, z3]])


elif NNODE == 4:

    # physical_coords = {'P1': (3, 0, 0),
    #                    'P2': (4, 0, 0),
    #                    'P3': (4, 0, 1),
    #                    'P4': (3, 0, 1),
    #                    }  # example coordinates for the nodes
    # physical_coords = {'P1': (0.11, 0, 0),
    #                    'P2': (0.20, 0, 0),
    #                    'P3': (0.20, 0, 0.04),
    #                    'P4': (0.11, 0, 0.04),
    #                    }  # example coordinates for the nodes

    # cylinder, di=1000mm, h=1000mm, t=10 mm
    physical_coords = {'P1': (1000, 0, 0),
                       'P2': (1020, 0, 0),
                       'P3': (1020, 0, 1000),
                       'P4': (1000, 0, 1000),
                       }  # example coordinates for the nodes

    midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0] + physical_coords['P4'][0]) / 4, \
                (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1] + physical_coords['P4'][1]) / 4, \
        (physical_coords['P1'][2] + physical_coords['P2'][2] + physical_coords['P3'][2] + physical_coords['P4'][2]) / 4

    physical_coords_map = {'r1': physical_coords['P1'][0], 'th1': physical_coords['P1'][1], 'z1': physical_coords['P1'][2],
                           'r2': physical_coords['P2'][0], 'th2': physical_coords['P2'][1], 'z2': physical_coords['P2'][2],
                           'r3': physical_coords['P3'][0], 'th3': physical_coords['P3'][1], 'z3': physical_coords['P3'][2],
                           'r4': physical_coords['P4'][0], 'th4': physical_coords['P4'][1], 'z4': physical_coords['P4'][2],
                           }
    # Define the physical nodes as a matrix so it can be used a symbolic evaluation
    physical_nodes = sp.Matrix([[r1, th1, z1],
                                [r2, th2, z2],
                                [r3, th3, z3],
                                [r4, th4, z4]])
elif NNODE == 8:

    def midpoint(p1, p2):
        return (p1[0] + p2[0]) / 2, 0, (p1[2] + p2[2]) / 2

    # physical_coords = {'P1': (3, 0, 0),
    #                    'P2': (4, 0, 0),
    #                    'P3': (4, 0, 1),
    #                    'P4': (3, 0, 1),
    #                    }  # example coordinates for the nodes
    # physical_coords = {'P1': (0.11, 0, 0),
    #                    'P2': (0.20, 0, 0),
    #                    'P3': (0.20, 0, 0.04),
    #                    'P4': (0.11, 0, 0.04),
    #                    }  # example coordinates for the nodes

    # cylinder, di=1000mm, h=1000mm, t=10 mm
    physical_coords = {'P1': (1000, 0, 0),
                       'P2': (1020, 0, 0),
                       'P3': (1020, 0, 1000),
                       'P4': (1000, 0, 1000),
                       }  # example coordinates for the nodes

    physical_coords['P5'] = midpoint(physical_coords['P1'], physical_coords['P2'])
    physical_coords['P6'] = midpoint(physical_coords['P2'], physical_coords['P3'])
    physical_coords['P7'] = midpoint(physical_coords['P3'], physical_coords['P4'])
    physical_coords['P8'] = midpoint(physical_coords['P4'], physical_coords['P1'])

    physical_coords_map = {'r1': physical_coords['P1'][0], 'th1': physical_coords['P1'][1], 'z1': physical_coords['P1'][2],
                           'r2': physical_coords['P2'][0], 'th2': physical_coords['P2'][1], 'z2': physical_coords['P2'][2],
                           'r3': physical_coords['P3'][0], 'th3': physical_coords['P3'][1], 'z3': physical_coords['P3'][2],
                           'r4': physical_coords['P4'][0], 'th4': physical_coords['P4'][1], 'z4': physical_coords['P4'][2],
                           'r5': physical_coords['P5'][0], 'th5': physical_coords['P5'][1], 'z5': physical_coords['P5'][2],
                           'r6': physical_coords['P6'][0], 'th6': physical_coords['P6'][1], 'z6': physical_coords['P6'][2],
                           'r7': physical_coords['P7'][0], 'th7': physical_coords['P7'][1], 'z7': physical_coords['P7'][2],
                           'r8': physical_coords['P8'][0], 'th8': physical_coords['P8'][1], 'z8': physical_coords['P8'][2],
                           }

    # Define the physical nodes as a matrix so it can be used a symbolic evaluation
    physical_nodes = sp.Matrix([[r1, th1, z1],
                                [r2, th2, z2],
                                [r3, th3, z3],
                                [r4, th4, z4],
                                [r5, th5, z5],
                                [r6, th6, z6],
                                [r7, th7, z7],
                                [r8, th8, z8],

                                ])

else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

import pprint
pprint.pprint(physical_coords)

{'P1': (1000, 0, 0),
 'P2': (1020, 0, 0),
 'P3': (1020, 0, 1000),
 'P4': (1000, 0, 1000)}


In [44]:
# Define the shape functions symbolically
def shape_functions() -> sp.Matrix:
    """shape functions"""

    if NNODE == 3:
        return sp.Matrix([
            1 - R - Z,
            R,
            Z,
        ])
    elif NNODE == 4:
        return sp.Matrix([
            0.25 * (1 - R) * (1 - Z),
            0.25 * (1 + R) * (1 - Z),
            0.25 * (1 + R) * (1 + Z),
            0.25 * (1 - R) * (1 + Z),
        ])
    elif NNODE == 8:
        return sp.Matrix([
            0.25 * (1 - R) * (1 - Z) * (-R - Z -1),
            0.25 * (1 + R) * (1 - Z) * (R - Z -1),
            0.25 * (1 + R) * (1 + Z) * (R + Z -1),
            0.25 * (1 - R) * (1 + Z) * (-R + Z -1),

            0.5 * (1 - R ** 2) * (1 - Z),  # 5
            0.5 * (1 + R) * (1 - Z ** 2),  # 6
            0.5 * (1 - R ** 2) * (1 + Z),  # 7
            0.5 * (1 - R) * (1 - Z ** 2),  # 8

        ])
    else:
        raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")


# checking the shape functions.
if NNODE == 3:
    coords = (
        (0, 0),  # 1
        (1, 0),  # 2
        (0, 1)   # 3
    )
elif NNODE == 4:
    coords = (
        (-1, -1),  # 1
        (1, -1),  # 2
        (1, 1),  # 3
        (-1, 1)  # 4
    )
elif NNODE == 8:
    coords = (
    (-1, -1),  # 1
    (1, -1),  # 2
    (1, 1),  # 3
    (-1, 1),  # 4
    (0, -1),  # 5
    (1, 0),  # 6
    (0, 1),  # 7
    (-1, 0)  # 8
    )
else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

sh = shape_functions()  # N1, N2, N3, N4, N5, N6, N7, N8
for cindex, coord in enumerate(coords):
    s, z = coord
    for i in range(NNODE):
        if i != cindex:
            assert np.isclose(sh.subs({R: s, Z: z})[i], 0)
        else:
            assert np.isclose(sh.subs({R: s, Z: z})[i], 1)



def isoparametric_mapping() -> np.array:
    """
    Isoparametric mapping natural -> physical coordinates.

    :return: functions returning the physical coordinates
    """

    sh = shape_functions()  # shape functions with substituted values

    # return: u, v mapping. theta is not included as it is not used in this mapping
    return sp.Matrix([sp.Matrix(sh).dot(physical_nodes[:, 0]), sp.Matrix(sh).dot(physical_nodes[:, 2])])


# Calculate the derivatives by r and z (u, v), symbolically
def shape_function_derivatives() -> np.array:
    """
    Shape function derivatives.
    [0] = dNi/dr, [1] = dNi/dz
    """
    sh = shape_functions()  # N1, N2, N3, N4
    dN_dr = []
    dN_dz = []

    for i in range(len(sh)):
        dN_dr.append(sp.diff(sh[i], R))
        dN_dz.append(sp.diff(sh[i], Z))

    return dN_dr, dN_dz


print()
print('The shape functions are:')
print(shape_functions())
print()
print('The derivatives are:')
derivatives = shape_function_derivatives()
for i, dN in enumerate(derivatives[0]):
    print(f'dN{i + 1}/ds = {dN}')
for i, dN in enumerate(derivatives[1]):
    print(f'dN{i + 1}/dt = {dN}')

print()
print('The isoparametric mapping is:')
x_map = isoparametric_mapping()[0]
y_map = isoparametric_mapping()[1]
print(f'x = {x_map}')
print(f'y = {y_map}')

# Substitute the symbolic coordinates with numerical values
st_values = {
    R: 0.3333,  # example value for s
    Z: 0.3333,  # example value for t
}
print()
print('The isoparametric mapping with coordinate values substituted is:')
x_map_substituted = x_map.subs(st_values)
y_map_substituted = y_map.subs(st_values)
print(f'x = {x_map_substituted}')
print(f'y = {y_map_substituted}')

print()
print('The derivatives of the shape functions by s are:')
print(derivatives[0])  # derivatives by s

print()
print('The derivatives of the shape functions by t are:')
print(derivatives[1])  # derivatives by t



The shape functions are:
Matrix([[(0.25 - 0.25*R)*(1 - Z)], [(1 - Z)*(0.25*R + 0.25)], [(0.25*R + 0.25)*(Z + 1)], [(0.25 - 0.25*R)*(Z + 1)]])

The derivatives are:
dN1/ds = 0.25*Z - 0.25
dN2/ds = 0.25 - 0.25*Z
dN3/ds = 0.25*Z + 0.25
dN4/ds = -0.25*Z - 0.25
dN1/dt = 0.25*R - 0.25
dN2/dt = -0.25*R - 0.25
dN3/dt = 0.25*R + 0.25
dN4/dt = 0.25 - 0.25*R

The isoparametric mapping is:
x = r1*(0.25 - 0.25*R)*(1 - Z) + r2*(1 - Z)*(0.25*R + 0.25) + r3*(0.25*R + 0.25)*(Z + 1) + r4*(0.25 - 0.25*R)*(Z + 1)
y = z1*(0.25 - 0.25*R)*(1 - Z) + z2*(1 - Z)*(0.25*R + 0.25) + z3*(0.25*R + 0.25)*(Z + 1) + z4*(0.25 - 0.25*R)*(Z + 1)

The isoparametric mapping with coordinate values substituted is:
x = 0.1111222225*r1 + 0.2222277775*r2 + 0.4444222225*r3 + 0.2222277775*r4
y = 0.1111222225*z1 + 0.2222277775*z2 + 0.4444222225*z3 + 0.2222277775*z4

The derivatives of the shape functions by s are:
[0.25*Z - 0.25, 0.25 - 0.25*Z, 0.25*Z + 0.25, -0.25*Z - 0.25]

The derivatives of the shape functions by t are:
[0.25*

### The shape function matrix

The shape function matrix $N$ is a matrix that contains the shape functions for each node. It has the structure:
$$
N = \begin{bmatrix}
N_i & 0 & 0 & ... & N_m & 0 & 0 \\
0 & N_i & 0 & ... & 0 & N_m & 0 \\
0 & 0 & N_i & ... & 0 & 0 & N_m \\
\end{bmatrix}
$$

In [45]:
def N_matrix() -> np.array:
    """
    shape function matrix N.

    """
    _N = shape_functions()  # the shape functions

    # preparing the return matrix
    _ret = sp.Matrix(ND, ND * NNODE, lambda i, j: 0)  # ND rows, ND * NNODE columns

    # filling the matrix
    for i in range(NNODE):
        for j in range(ND):
            _ret[j, j + ND * i] = _N[i]

    return _ret


_N = N_matrix()
print("The shape function matrix N is {}:".format(_N.shape))
print()
print('The shape function matrix N for this element:')
_N_e = _N.subs(physical_coords_map)
sp.pprint(_N_e)

print('\nThe shape function matrix N for this element at node 1:')
_N_e_node1 = _N_e.subs({'R': physical_coords['P1'][0], 'Z': physical_coords['P1'][2]})
sp.pprint(_N_e_node1.applyfunc(lambda x: x.round(4)))

# summing each row of the matrix
for i in range(ND):
    print(f'Sum of row {i + 1} of the shape function matrix N: {sum(_N_e_node1[i, :].applyfunc(lambda x: x.round(4)))}')


The shape function matrix N is (3, 12):

The shape function matrix N for this element:
⎡(0.25 - 0.25⋅R)⋅(1 - Z)             0                        0             (1 ↪
⎢                                                                              ↪
⎢           0             (0.25 - 0.25⋅R)⋅(1 - Z)             0                ↪
⎢                                                                              ↪
⎣           0                        0             (0.25 - 0.25⋅R)⋅(1 - Z)     ↪

↪  - Z)⋅(0.25⋅R + 0.25)             0                        0             (0. ↪
↪                                                                              ↪
↪          0             (1 - Z)⋅(0.25⋅R + 0.25)             0                 ↪
↪                                                                              ↪
↪          0                        0             (1 - Z)⋅(0.25⋅R + 0.25)      ↪

↪ 25⋅R + 0.25)⋅(Z + 1)             0                        0             (0.2 ↪
↪                   

### Jacobian matrix

In [46]:
def jacobian() -> sp.Matrix:
    """
    Calculate the jacobian matrix for the isoparametric mapping.
    """
    # The isoparametric mapping functions r(R,Z) and z(R,Z)
    mapping = isoparametric_mapping()
    r_map = mapping[0]
    z_map = mapping[1]

    # Calculate the derivatives
    dr_dR = sp.diff(r_map, R)
    dr_dZ = sp.diff(r_map, Z)
    dz_dR = sp.diff(z_map, R)
    dz_dZ = sp.diff(z_map, Z)

    return sp.Matrix([[dr_dR, dr_dZ],
                      [dz_dR, dz_dZ]])


# Calculate the jacobian matrix symbolically
J = jacobian()

# Substitute the symbolic coordinates with numerical values in the Jacobian
J_element = J.subs(physical_coords_map)

print('The jacobian matrix with coordinate values substituted is:')
sp.pprint(J_element)
print()

# Evaluate the jacobian matrix at some points
for r_eval in [-1, 1]:
    for z_eval in [-1, 1]:
        J_evaluated = J_element.subs({R: r_eval, Z: z_eval})
        print(f'Jacobian matrix at r={r_eval}, z={z_eval}:')
        sp.pprint(J_evaluated)
        print(f'The determinant at this point is |J|={J_evaluated.det():.2f}')
        print()

The jacobian matrix with coordinate values substituted is:
⎡10.0    0  ⎤
⎢           ⎥
⎣ 0    500.0⎦

Jacobian matrix at r=-1, z=-1:
⎡10.0    0  ⎤
⎢           ⎥
⎣ 0    500.0⎦
The determinant at this point is |J|=5000.00

Jacobian matrix at r=-1, z=1:
⎡10.0    0  ⎤
⎢           ⎥
⎣ 0    500.0⎦
The determinant at this point is |J|=5000.00

Jacobian matrix at r=1, z=-1:
⎡10.0    0  ⎤
⎢           ⎥
⎣ 0    500.0⎦
The determinant at this point is |J|=5000.00

Jacobian matrix at r=1, z=1:
⎡10.0    0  ⎤
⎢           ⎥
⎣ 0    500.0⎦
The determinant at this point is |J|=5000.00



### The general $\begin{bmatrix}B\end{bmatrix}$ matrix

In [47]:
def calculate_B_general(harmonic_l: int) -> tuple[sp.Matrix, sp.Matrix]:
    """
    Calculates the symbolic strain-displacement matrices B_bar and B_double_bar for a given harmonic.

    This function is designed to be used with the SymPy library. It takes symbolic
    expressions for coordinates and shape functions and returns symbolic SymPy matrices.
    This directly implements the relationships defined around Eq. (14) and explicitly
    shown for B_bar_in in Eq. (15) of the provided text.

    B_bar is for the SYMMETRIC part of the load/displacement.
    B_double_bar is for the ANTI-SYMMETRIC part.

    Every load, for every harmonic n, can be split into these two parts. They are completely independent and don't interact with each other.

    +----------+------------------------------------+---------------------------------------+
    | Harmonic | B_bar (Symmetric Part)             | B_double_bar (Anti-Symmetric Part)    |
    +==========+====================================+=======================================+
    | n = 0    | Classic Axisymmetric Case          | Pure Torsion                          |
    |          | (Internal pressure, spinning)      | (Twisting load)                       |
    |          | Displacements: u, w                | Displacement: v                       |
    +----------+------------------------------------+---------------------------------------+
    | n = 1    | Bending                            | Transverse Shear                      |
    |          | (Loads that vary like cos(θ))      | (Loads that vary like sin(θ))         |
    |          | Displacements: u, w                | Displacement: v                       |
    +----------+------------------------------------+---------------------------------------+
    | n > 1    | Higher-Order Symmetric Loads       | Higher-Order Anti-Symmetric Loads     |
    |          | (e.g., Ovalization, cos(2θ) loads) | (e.g., sin(2θ) loads)                 |
    +----------+------------------------------------+---------------------------------------+

    Args:
        harmonic_l (int): The harmonic number (a standard Python integer).

    Returns:
        tuple[sympy.Matrix, sympy.Matrix]: A tuple containing:
            - B_bar (sympy.Matrix): The symbolic symmetric strain-displacement matrix, shape (6, 3*M).
            - B_double_bar (sympy.Matrix): The symbolic antisymmetric strain-displacement matrix, shape (6, 3*M).
    """

    # --- Get symbolic components from helper functions (your existing workflow) ---
    det_J = J_element.det()
    dN_dR, dN_dZ = shape_function_derivatives()

    # Derivatives of the r, z mapping w.r.t natural coordinates R, Z
    dzdZ = sp.diff(y_map, Z).subs(physical_coords_map)
    drdZ = sp.diff(x_map, Z).subs(physical_coords_map)
    dzdR = sp.diff(y_map, R).subs(physical_coords_map)
    drdR = sp.diff(x_map, R).subs(physical_coords_map)

    # Get symbolic expressions for the radius 'r' and shape functions 'N'
    r_iso = isoparametric_mapping()[0].subs(physical_coords_map)
    N_funcs = shape_functions().subs(physical_coords_map)

    B_bar = sp.zeros(6, ND * NNODE)
    B_double_bar = sp.zeros(6, ND * NNODE)

    # --- Loop through each node to build the matrix columns ---
    for i in range(NNODE):
        # Calculate shape function derivatives w.r.t. physical coords (r, z)
        # This is the chain rule, implemented just like in your example.
        dNi_dr = (1 / det_J) * (dzdZ * dN_dR[i] - drdZ * dN_dZ[i])
        dNi_dz = (1 / det_J) * (drdR * dN_dZ[i] - dzdR * dN_dR[i])
        Ni = N_funcs[i]

        B_bar_i = sp.zeros(6, ND)
                # Row 1: epsilon_r = du/dr
        B_bar_i[0, 0] = dNi_dr
        # Row 2: epsilon_theta = u/r + (1/r) * dv/d(theta)
        B_bar_i[1, 0] = Ni / r_iso
        B_bar_i[1, 1] = harmonic_l * Ni / r_iso
        # Row 3: epsilon_z = dw/dz
        B_bar_i[2, 2] = dNi_dz
        # Row 4: gamma_r_theta = (1/r)*du/d(theta) + dv/dr - v/r
        B_bar_i[3, 0] = -harmonic_l * Ni / r_iso
        B_bar_i[3, 1] = dNi_dr - Ni / r_iso
        # Row 5: gamma_theta_z = (1/r)*dw/d(theta) + dv/dz
        B_bar_i[4, 1] = dNi_dz
        B_bar_i[4, 2] = -harmonic_l * Ni / r_iso
        # Row 6: gamma_z_r = du/dz + dw/dr
        B_bar_i[5, 0] = dNi_dz
        B_bar_i[5, 2] = dNi_dr

        # Place this 6x3 submatrix into the correct columns of the full B_bar
        B_bar[:, 3 * i : 3 * (i + 1)] = B_bar_i


        B_double_bar_i = sp.zeros(6, ND)

        # Row 1: epsilon_r
        B_double_bar_i[0, 0] = dNi_dr
        # Row 2: epsilon_theta
        B_double_bar_i[1, 0] = Ni / r_iso
        B_double_bar_i[1, 1] = -harmonic_l * Ni / r_iso # Sign flip
        # Row 3: epsilon_z
        B_double_bar_i[2, 2] = dNi_dz
        # Row 4: gamma_r_theta
        B_double_bar_i[3, 0] = harmonic_l * Ni / r_iso  # Sign flip
        B_double_bar_i[3, 1] = dNi_dr - Ni / r_iso
        # Row 5: gamma_theta_z
        B_double_bar_i[4, 1] = dNi_dz
        B_double_bar_i[4, 2] = harmonic_l * Ni / r_iso # Sign flip
        # Row 6: gamma_z_r
        B_double_bar_i[5, 0] = dNi_dz
        B_double_bar_i[5, 2] = dNi_dr

        B_double_bar[:, 3 * i : 3 * (i + 1)] = B_double_bar_i

    return B_bar, B_double_bar

B_bar, B_double_bar = calculate_B_general(harmonic_l=0)
sp.pprint(B_bar.subs({R: -0.5774, Z: 0.5774})[:, :3])  # Show only the first 3 columns (u', v', w')

print()

B_bar, B_double_bar = calculate_B_general(harmonic_l=1)
sp.pprint(B_double_bar.subs({R: -0.5774, Z: 0.5774})[:, :3])  # Show only the first 3 columns (u', v', w')


⎡     -0.010565                0               0     ⎤
⎢                                                    ⎥
⎢0.00016595100106948           0               0     ⎥
⎢                                                    ⎥
⎢         0                    0           -0.0007887⎥
⎢                                                    ⎥
⎢         0           -0.0107309510010695      0     ⎥
⎢                                                    ⎥
⎢         0               -0.0007887           0     ⎥
⎢                                                    ⎥
⎣    -0.0007887                0           -0.010565 ⎦

⎡     -0.010565                0                     0         ⎤
⎢                                                              ⎥
⎢0.00016595100106948  -0.00016595100106948           0         ⎥
⎢                                                              ⎥
⎢         0                    0                -0.0007887     ⎥
⎢                                                              ⎥
⎢0.0

### The constitutive matrix $\begin{bmatrix}D\end{bmatrix}$

The constitutive matrix relates the stress vector to the strain vector. For isotropic material it is given by:
$$
\begin{bmatrix}D\end{bmatrix} = \frac{E}{(1 - 2\nu)(1 + \nu)} \begin{bmatrix}
1 - \nu & \nu & \nu & 0 & 0 & 0 \\
\nu & 1 - \nu & \nu & 0 & 0 & 0 \\
\nu & \nu & 1 - \nu & 0 & 0 & 0 \\
0 & 0 & 0 & \frac{1 - 2\nu}{2} & 0 & 0\\
0 & 0 & 0 & 0 & \frac{1 - 2\nu}{2} & 0\\
0 & 0 & 0 & 0 & 0 & \frac{1 - 2\nu}{2}
\end{bmatrix}
$$


In [48]:
def D_matrix():
    """
    The constitutive matrix for the general axissymmetric element.
    """
    D = sp.zeros(6, 6)
    D[0, 0] = 1 - nu
    D[0, 1] = nu
    D[0, 2] = nu

    D[1, 0] = nu
    D[1, 1] = 1 - nu
    D[1, 2] = nu

    D[2, 0] = nu
    D[2, 1] = nu
    D[2, 2] = 1 - nu

    D[3, 3] = D[4, 4] = D[5, 5] = (1 - 2 * nu) / 2

    D *= E / ((1 - 2 * nu) * (1 + nu))

    return D

D = D_matrix()
sp.pprint(D.applyfunc(lambda x: round(x / 1e9, 2)))

⎡269.23  115.38  115.38    0      0      0  ⎤
⎢                                           ⎥
⎢115.38  269.23  115.38    0      0      0  ⎥
⎢                                           ⎥
⎢115.38  115.38  269.23    0      0      0  ⎥
⎢                                           ⎥
⎢  0       0       0     76.92    0      0  ⎥
⎢                                           ⎥
⎢  0       0       0       0    76.92    0  ⎥
⎢                                           ⎥
⎣  0       0       0       0      0    76.92⎦


### Quadrature points and weights

In [49]:
if NNODE == 3:
    gi_data = {
        1: {
            'point': ((1/3, 1/3),),  # midpoint of the triangle
            'weight': (1,)
        },
        2: {
            # 'point': [-1 / np.sqrt(3), 1 / np.sqrt(3)],
            'point': [-0.5774, 0.5774],
            'weight': [1, 1]
        },
        3: {
            'point': ((1/6, 1/6), (4/6, 1/6), (1/6, 4/6), ),
            'weight': (1/3, 1/3, 1/3)
        },
    }
elif NNODE in (4, 8):
    gi_data = {
        1: {
            'point': [0],
            'weight': [2]
        },
        2: {
            # 'point': [-1 / np.sqrt(3), 1 / np.sqrt(3)],
            'point': [-0.5774, 0.5774],
            'weight': [1, 1]
        },
        3: {
            # 'point': [-np.sqrt(3 / 5), 0, np.sqrt(3 / 5)],
            'point': [-0.7745966, 0, 0.7745966],
            'weight': [5 / 9,  8 / 9, 5 / 9]
        },
        4: {
            'point': [-0.861136, -0.339981, 0.339981, 0.861136],
            'weight': [0.347855, 0.652145, 0.652145, 0.347855]
        },
        5: {
            'point': [-0.906180, -0.538469, 0, 0.906180, 0.538469],
            'weight': [0.236927, 0.478629, 0.568889, 0.478629, 0.236927]
        },
        6: {
            'point': [-0.932469, -0.661209, -0.238619, 0.238619, 0.661209, 0.932469],
            'weight': [0.171324, 0.360761, 0.467914, 0.467914, 0.360761, 0.171324]
        }
    }
else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

### The stiffness matrix from the general B matrix

In [50]:
def K_general(harmonic_l: int, ng: int) -> tuple:
    """
    Calculates the stiffness kernel integrated over the r-z plane at a
    specific angle theta.

    This function computes the result of the integral:
    ∫[B(θ)]^T [D] [B(θ)] * r dr dz
    at the given, fixed angle theta.

    Args:
        harmonic_l (int): The harmonic number (l) for the Fourier expansion.
        ng (int): The number of Gauss points to use for the r-z integration.

    Returns:
        np.array: The numerical stiffness kernel matrix.
    """
    # --- Step 1: Get the symbolic B and D matrices ---
    # B is now evaluated at a specific numerical theta, but remains symbolic
    # in terms of the natural coordinates R and Z.
    B_bar, B_double_bar = calculate_B_general(harmonic_l)
    D = D_matrix()

    # --- Step 2: Form the r-z integrand kernels ---
    # This is the part of the stiffness integral that needs to be
    # integrated numerically over the r-z plane.
    K_bar_kernel_rz = (B_bar.T * D * B_bar).subs(physical_coords_map)
    K_double_bar_kernel_rz = (B_double_bar.T * D * B_double_bar).subs(physical_coords_map)

    # --- Step 3: Determine the constant factor from the theta integration ---
    if harmonic_l == 0:
        theta_integral_factor = 2 * np.pi
    else:
        theta_integral_factor = np.pi

    # --- Step 4: Perform Gauss Quadrature over the r-z plane ---
    num_dof = NNODE * ND
    K_bar_numeric = np.zeros((num_dof, num_dof))
    K_double_bar_numeric = np.zeros((num_dof, num_dof))

    r_iso = isoparametric_mapping()[0].subs(physical_coords_map)

    if NNODE in (4, 8): # For quadrilateral elements
        for i in range(ng):
            for j in range(ng):
                s = gi_data[ng]['point'][i]  # R-direction Gauss point
                t = gi_data[ng]['point'][j]  # Z-direction Gauss point
                w = gi_data[ng]['weight'][i] * gi_data[ng]['weight'][j]

                gp_map = {R: s, Z: t}

                # Evaluate Jacobian determinant and radius at the Gauss point
                det_J_val = float(J_element.subs(gp_map).det().evalf())
                r_val = float(r_iso.subs(gp_map).evalf())

                # Evaluate the r-z kernels at the Gauss point
                K_bar_kernel_val = np.array(K_bar_kernel_rz.subs(gp_map).evalf(), dtype=float)
                K_double_bar_kernel_val = np.array(K_double_bar_kernel_rz.subs(gp_map).evalf(), dtype=float)

                # Calculate the contribution for each matrix
                # This is the integrand evaluated at the Gauss point, multiplied by all factors
                # Integrand = [Kernel] * r * detJ * weight * theta_factor
                contribution_bar = K_bar_kernel_val * r_val * det_J_val * w * theta_integral_factor
                contribution_double_bar = K_double_bar_kernel_val * r_val * det_J_val * w * theta_integral_factor

                K_bar_numeric += contribution_bar
                K_double_bar_numeric += contribution_double_bar
    else:
        raise ValueError("Unsupported number of nodes. Only 4 and 8 are shown here.")

    return K_bar_numeric, K_double_bar_numeric

sp.pprint(K_general(0, NGS[1])[0])

 [[ 2.82422523e+16  0.00000000e+00  3.01673171e+14 -2.84734565e+16 
    0.00000000e+00  6.08182892e+13 -1.42373293e+16  0.00000000e+00 
  -3.04090197e+14  1.41144443e+16  0.00000000e+00 -5.84012631e+13] 
  [ 0.00000000e+00  8.30169131e+15  0.00000000e+00  0.00000000e+00 
   -8.13411110e+15  0.00000000e+00  0.00000000e+00 -4.06897053e+15 
   0.00000000e+00  0.00000000e+00  4.14545196e+15  0.00000000e+00] 
  [ 3.01673171e+14  0.00000000e+00  8.14760423e+15 -6.12205024e+13 
    0.00000000e+00 -8.13057520e+15 -3.06103761e+14  0.00000000e+00 
  -4.07330396e+15  5.84012631e+13  0.00000000e+00  4.05627492e+15] 
  [-2.84734565e+16  0.00000000e+00 -6.12205024e+13  2.87255904e+16 
    0.00000000e+00 -3.08520787e+14  1.43560338e+16  0.00000000e+00 
   6.36375284e+13 -1.42373293e+16  0.00000000e+00  3.06103761e+14] 
  [ 0.00000000e+00 -8.13411110e+15  0.00000000e+00  0.00000000e+00 
    7.97948449e+15  0.00000000e+00  0.00000000e+00  3.98432103e+15 
   0.00000000e+00  0.00000000e+00 -4.06897053e+1

# Loads

### The Load Vector $\begin{bmatrix}F\end{bmatrix}$

The next step is to formulate the **consistent nodal load vector** for each harmonic, $\{F_n\}$.

Just as we decomposed the displacement response into a Fourier series, the applied external load must also be decomposed in the same way.

***

### ## 1. Load Decomposition

You must first represent your specific applied load (e.g., wind pressure, a point force) as a Fourier series in the circumferential ($\theta$) direction. For a general surface pressure $\vec{p}$ with components $(p_r, p_\theta, p_z)$, this looks like:
$$p_r(r, \theta, z) = \sum_{n=0}^{\infty} p_{rn}(r, z) \cos(n\theta)$$$$p_\theta(r, \theta, z) = \sum_{n=0}^{\infty} p_{\theta n}(r, z) \sin(n\theta)$$$$p_z(r, \theta, z) = \sum_{n=0}^{\infty} p_{zn}(r, z) \cos(n\theta)$$
You would perform a Fourier analysis on your load to find the amplitude functions ($p_{rn}, p_{\theta n}, p_{zn}$). For many common loads, these are standard or simple to derive.

***

### ## 2. Nodal Force Vector Formulation

Once you have the load amplitudes for each harmonic $n$, you can calculate the corresponding consistent nodal force vector, $\{F_n\}$. This is done using the shape functions $[N]$ and the principle of virtual work. The formula for the element nodal force vector is:
$$\{F_n\}_e = \int_A [N]^T \{p_n\} dA$$
Where $\{p_n\}$ is the vector of load amplitudes $\{p_{rn}, p_{\theta n}, p_{zn}\}^T$ for that harmonic. Just like with the stiffness matrix, the integral is taken over the 2D domain, and the circumferential part is handled analytically.

***

### ## 3. Assembly and Solution ⚙️

With all the components ready, the final procedure is:
1.  **Choose Harmonics:** Decide which harmonics ($n=0, 1, 2, ...$) are significant based on your load decomposition.
2.  **For each harmonic $n$**:
    * Assemble the global stiffness matrix $[K_n]$ from the element matrices $[k_e]_n$.
    * Assemble the global force vector $\{F_n\}$ from the element force vectors $\{F_n\}_e$.
    * Solve the system of linear equations: $[K_n]\{u_n\} = \{F_n\}$ to find the nodal displacement amplitudes $\{u_n\}$.
3.  **Superimpose Results:** The final displacement is the sum of the solutions from all the calculated harmonics. The stresses are calculated similarly and also summed.

### Hydrostatic Pressure Load Calculation

In [51]:
def get_hydrostatic_pressure_load(
    r: float,
    z: float,
    inner_radius: float,
    fluid_level_H: float,
    fluid_gamma: float
) -> np.array:
    """
    Calculates the load vector (pr, p_theta, pz) for hydrostatic pressure.

    Args:
        r: The radial coordinate of the point.
        z: The vertical coordinate of the point.
        inner_radius: The inner radius of the cylinder.
        fluid_level_H: The vertical coordinate of the fluid's free surface.
        fluid_gamma: The specific weight of the fluid (e.g., density * g).
    """
    p_r = 0.0
    p_theta = 0.0
    p_z = 0.0

    # Check if the point is on the inner surface AND below the fluid level
    if np.isclose(r, inner_radius) and z < fluid_level_H:
        # Calculate the depth of the point below the fluid surface
        depth = fluid_level_H - z

        # Calculate the hydrostatic pressure
        pressure = fluid_gamma * depth

        p_r = pressure

    return np.array([p_r, p_theta, p_z]).reshape(-1, 1)

### Assembling the Element Force Vector

In [52]:
def get_hydrostatic_load_force_vector():

    # Initialize the element force vector
    F_element = np.zeros((ND * NNODE, 1))

    if NNODE == 3:
        # loaded nodes are: 1, 3

        # for this the distance between nodes P1 and P4
        pts = physical_coords
        L_edge = np.sqrt((pts['P1'][0] - pts['P3'][0])**2 + (pts['P1'][2] - pts['P3'][2])**2)
        print('L_edge', L_edge)
        det_J_edge = L_edge / 2.0
        print('det_J_edge', det_J_edge)

        # Use 1 Gauss points for the line integral
        gauss_points = gi_data[2]['point']
        gauss_weights = gi_data[2]['weight']

        for i in range(len(gauss_points)):
            s_gp = gauss_points[i]
            w_gp = gauss_weights[i]

            # Evaluate 1D shape functions at the Gauss point s_gp
            N3_at_gp = 0.5 * (1 - s_gp)
            N1_at_gp = 0.5 * (1 + s_gp)

            # 2. Find the physical coordinates (r, z) of the Gauss point
            #    by interpolating the nodal coordinates (r1, z1) and (r4, z4)
            r_gp = float((N1_at_gp * r1 + N3_at_gp * r3).subs(physical_coords_map).evalf())
            z_gp = float((N1_at_gp * z1 + N3_at_gp * z3).subs(physical_coords_map).evalf())

            # 3. Calculate the pressure vector at this Gauss point
            # p_vector is a (3, 1) vector: [p_r, p_theta, p_z]
            p_vector = get_hydrostatic_pressure_load(
                r=float(r_gp),
                z=float(z_gp),
                inner_radius=float(r_gp),
                fluid_level_H=100000.0,  # Fluid level height
                fluid_gamma=1e-5  # 9.81e-6,     # Specific weight of water in N/mm3
            )

            # Construct the 3x9 [N] matrix for this point on the edge
            # Only columns for nodes 1 and 3 will be non-zero
            N_matrix_at_gp = np.zeros((3, 9))
            # Node P3 contributions
            N_matrix_at_gp[0, 0] = N3_at_gp
            N_matrix_at_gp[1, 1] = N3_at_gp
            N_matrix_at_gp[2, 2] = N3_at_gp
            # Node P1 contributions
            N_matrix_at_gp[0, 6] = N1_at_gp
            N_matrix_at_gp[1, 7] = N1_at_gp
            N_matrix_at_gp[2, 8] = N1_at_gp

            sp.pprint(N_matrix_at_gp)

            # Calculate the contribution at this Gauss point
            F_contribution = 2 * np.pi * r_gp * (N_matrix_at_gp.T @ p_vector) * det_J_edge * w_gp

            # Add to the total element force vector
            F_element += F_contribution

        # F_element now holds the consistent nodal forces for the element.
        return F_element

    elif NNODE == 4:

        # loaded nodes are: 1, 4

        # for this the distance between nodes P1 and P4
        pts = physical_coords
        L_edge = np.sqrt((pts['P1'][0] - pts['P4'][0])**2 + (pts['P1'][2] - pts['P4'][2])**2)
        det_J_edge = L_edge / 2.0

        # Use 2 Gauss points for the line integral
        gauss_points = gi_data[2]['point']
        gauss_weights = gi_data[2]['weight']

        for i in range(len(gauss_points)):
            s_gp = gauss_points[i]
            w_gp = gauss_weights[i]

            # Evaluate 1D shape functions at the Gauss point s_gp
            N4_at_gp = 0.5 * (1 - s_gp)
            N1_at_gp = 0.5 * (1 + s_gp)

            # 2. Find the physical coordinates (r, z) of the Gauss point
            #    by interpolating the nodal coordinates (r1, z1) and (r4, z4)
            r_gp = float((N1_at_gp * r1 + N4_at_gp * r4).subs(physical_coords_map).evalf())
            z_gp = float((N1_at_gp * z1 + N4_at_gp * z4).subs(physical_coords_map).evalf())

            # 3. Calculate the pressure vector at this Gauss point
            # p_vector is a (3, 1) vector: [p_r, p_theta, p_z]
            p_vector = get_hydrostatic_pressure_load(
                r=float(r_gp),
                z=float(z_gp),
                inner_radius=float(r_gp),
                fluid_level_H=100000.0,  # Fluid level height
                fluid_gamma=1e-5  # 9.81e-6,     # Specific weight of water in N/mm3
            )

            print('p_vector.T:', p_vector.T)


            # Construct the 3x8 [N] matrix for this point on the edge
            # Only columns for nodes 1 and 2 will be non-zero
            N_matrix_at_gp = np.zeros((3, 12))
            # Node P1 contributions
            N_matrix_at_gp[0, 0] = N4_at_gp
            N_matrix_at_gp[1, 1] = N4_at_gp
            N_matrix_at_gp[2, 2] = N4_at_gp
            # Node P4 contributions
            N_matrix_at_gp[0, 9] = N1_at_gp
            N_matrix_at_gp[1, 10] = N1_at_gp
            N_matrix_at_gp[2, 11] = N1_at_gp

            # Calculate the contribution at this Gauss point
            F_contribution = 2 * np.pi * r_gp * (N_matrix_at_gp.T @ p_vector) * det_J_edge * w_gp

            # Add to the total element force vector
            F_element += F_contribution

        # F_element now holds the consistent nodal forces for the element.
        return F_element

    elif NNODE == 8:

        # Knoten auf der Kante: 1, 8, 4
        pts = physical_coords
        # Kantenlänge und Jacobi-Determinante
        L_edge = np.sqrt((pts['P1'][0] - pts['P4'][0])**2 + (pts['P1'][2] - pts['P4'][2])**2)
        det_J_edge = L_edge / 2.0

        # 3-Punkt-Gauß-Integration für die Kante
        gauss_points = [-np.sqrt(3/5), 0.0, np.sqrt(3/5)]
        gauss_weights = [5/9, 8/9, 5/9]

        # 1D-Shape-Funktionen für 3 Knoten
        def N1(s): return 0.5 * s * (s - 1)
        def N8(s): return (1 - s**2)
        def N4(s): return 0.5 * s * (s + 1)

        for s_gp, w_gp in zip(gauss_points, gauss_weights):
            # Interpolierte Koordinaten am Gauß-Punkt
            r_gp = float((N1(s_gp) * r1 + N8(s_gp) * r5 + N4(s_gp) * r4).subs(physical_coords_map))
            z_gp = float((N1(s_gp) * z1 + N8(s_gp) * z5 + N4(s_gp) * z4).subs(physical_coords_map))

            # Druckvektor am Gauß-Punkt
            p_vector = get_hydrostatic_pressure_load(
                r=r_gp,
                z=z_gp,
                inner_radius=r_gp,
                fluid_level_H=100000.0,
                fluid_gamma=1e-5
            )

            print('p_vector.T:', p_vector.T)

            # Shape-Funktionen am Gauß-Punkt
            N_vec = np.zeros((NNODE, 1))
            N_vec[0] = N1(s_gp)
            N_vec[7] = N8(s_gp)
            N_vec[3] = N4(s_gp)

            # Beitrag zum Kraftvektor
            for i in range(NNODE):
                for d in range(ND):
                    F_element[ND * i + d, 0] += N_vec[i, 0] * p_vector[d, 0] * 2 * np.pi * r_gp * det_J_edge * w_gp

        # F_element now holds the consistent nodal forces for the element.
        return F_element

    else:
        raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

print("Element Force Vector (F_element):")
sp.pprint(get_hydrostatic_load_force_vector())
print(sum(get_hydrostatic_load_force_vector()))

Element Force Vector (F_element):
p_vector.T: [[0.992113 0.       0.      ]]
p_vector.T: [[0.997887 0.       0.      ]]
[[3120647.80050989] 
 [      0.        ] 
 [      0.        ] 
 [      0.        ] 
 [      0.        ] 
 [      0.        ] 
 [      0.        ] 
 [      0.        ] 
 [      0.        ] 
 [3131121.5801338 ] 
 [      0.        ] 
 [      0.        ]]
p_vector.T: [[0.992113 0.       0.      ]]
p_vector.T: [[0.997887 0.       0.      ]]
[6251769.38064369]


# Solving the System of Equations

In [53]:
# Applying the boundary conditions.
def apply_boundary_conditions(K, F, fixed_dofs):
    """
    Applies boundary conditions to the stiffness matrix K and force vector F.
    Fixed nodes are removed from the system.
    """
    # Remove rows and columns corresponding to fixed nodes
    for dof in fixed_dofs:
        # setting the node row to zeros
        K[dof, :] = 0
        K[:, dof] = 0
        K[dof, dof] = 1  # Set diagonal to 1 for fixed nodes
        F[dof] = 0

    return K, F

# Solve the system of equations
def solve_system(K, F):
    """
    Solves the system of equations K * u = F for u.
    """
    # Convert to numpy arrays for numerical solving
    K_np = np.array(K).astype(np.float64)
    F_np = np.array(F).astype(np.float64)

    # Solve the linear system
    u = np.linalg.solve(K_np, F_np)

    return u

def get_reactionforces(K, F, u):
    """
    Calculates the reaction forces at the fixed nodes.
    """
    # Calculate the reaction forces
    reaction_forces = K @ u - F

    return reaction_forces

K = K_general(0, NGS[1])[0]  # frem the general B matrix
F = get_hydrostatic_load_force_vector()


if NNODE == 3:
    # fixed_dofs=[2, 5, 8, ]  # Fixed nodes for 3-node element
    fixed_dofs=[2, 5, ]  # open end
elif NNODE == 4:
    fixed_dofs=[2, 5, 8, 11]
    fixed_dofs=[2, 5]  # open end
elif NNODE == 8:
    fixed_dofs=[2, 5, 8, 11, 14, 20, 17, 23]  # Fixed nodes for 8-node element
    # fixed_dofs=[2, 5]  # open end
else:
    raise ValueError("Unsupported number of nodes. Only 4 or 8 nodes are supported")

K, F = apply_boundary_conditions(K, F, fixed_dofs=fixed_dofs)
u = solve_system(K, F)
# sp.pprint(u)
# print("\n")



p_vector.T: [[0.992113 0.       0.      ]]
p_vector.T: [[0.997887 0.       0.      ]]


# Getting the stress vector

In [54]:
ng = NGS[1]  # Number of Gauss points for the integration

def get_stress_vector(u, n, symm: bool = True):

    _D = D_matrix()
    if symm:
        Be = calculate_B_general(harmonic_l=n)[0]  # from the general B
    else:
        Be = calculate_B_general(harmonic_l=n)[1]  # from the double

    if NNODE == 3:
        # raise ValueError("This example is for a 4-noded quad element. Please use a 4-noded element.")

        for i in range(NGS[0]):
            for j in range(NGS[0]):
                s = gi_data[ng]['point'][i][0]
                t = gi_data[ng]['point'][j][0]

                B = Be.subs({R: s, Z: t}).evalf()

                epsilon_vector = B @ u  # Result is a (6, 1) vector

                # print()
                # print(f"Gauss point (s, t): {s}, {t}")
                # print('epsilon:', epsilon_vector)

                stress_vector = _D @ epsilon_vector # Result is a (6, 1) vector

                # print('sigma:', stress_vector)

    elif NNODE in (4, 8):

        for i in range(NGS[1]):
            for j in range(NGS[1]):
                s = gi_data[NGS[1]]['point'][i]
                t = gi_data[NGS[1]]['point'][j]

                B = Be.subs({R: s, Z: t}).evalf()

                epsilon_vector = B @ u  # Result is a (6, 1) vector

                print()
                print(f"Gauss point (s, t): {s}, {t}")
                # print('epsilon:', epsilon_vector)

                stress_vector = _D @ epsilon_vector # Result is a (6, 1) vector

                print('sigma:', stress_vector)
        #
        # # F_element now holds the consistent nodal forces for the element.
        # return F_element

    else:
        raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

    return stress_vector
print("Stresses from the internal pressure are:")
sp.pprint(get_stress_vector(u, n=0, symm=True))

Stresses from the internal pressure are:

Gauss point (s, t): -0.5774, -0.5774
sigma: Matrix([[-0.270892016486952], [50.1867625888512], [0.199328944019872], [0], [0], [0.0568809480859700]])

Gauss point (s, t): -0.5774, 0.5774
sigma: Matrix([[-0.273156278912916], [50.3205141838657], [0.238775143796435], [0], [0], [-0.0117840092934439]])

Gauss point (s, t): 0.5774, -0.5774
sigma: Matrix([[-0.703179305676615], [49.1808388457038], [-0.235704943464970], [0], [0], [0.0564638434639438]])

Gauss point (s, t): 0.5774, 0.5774
sigma: Matrix([[-0.706603307711285], [49.3118843816313], [-0.197418483297113], [0], [0], [-0.0122011139154622]])
⎡-0.706603307711285 ⎤
⎢                   ⎥
⎢ 49.3118843816313  ⎥
⎢                   ⎥
⎢-0.197418483297113 ⎥
⎢                   ⎥
⎢         0         ⎥
⎢                   ⎥
⎢         0         ⎥
⎢                   ⎥
⎣-0.0122011139154622⎦


### A harmonic loading: bending moment

In [59]:
def apply_bending_moment(M: float):

    # Apply a bending moment at node 1
    bending_moment = M  # kNm
    bending_moment *= 1e6  # Convert to Nmm for consistency with other units

    # The bending moment is applied as a force at the node
    # For an 8-node element, we apply it to the first node (node 1)
    F_element = np.zeros((ND * NNODE, 1))
    _R = (physical_coords['P3'][0] + physical_coords['P4'][0]) / 2  # Average radius of the element
    _F = -bending_moment / _R  # Convert to force per unit length

    print("Bending moment (M):", bending_moment)
    print("Physical radius at node 1 (r1):", _R)
    print("Bending moment (F_element):", _F)

    if NNODE == 3:
        raise ValueError('Not yet implemented for 3-node elements')

    elif NNODE == 4:
        # the force is applied in the z-direction over the nodes of the top edge.
        # the affected nodes are (n): 3, 4
        # the affected DOF indices are (3n-1): 8, 11
        F_element[8, 0] = (-bending_moment / physical_coords['P3'][0]) / 2  # Node 3
        F_element[11, 0] = (-bending_moment / physical_coords['P4'][0]) / 2  # Node 4

    else:

        # the force is applied in the z-direction over the nodes of the top edge.
        # the affected nodes are (n): 3, 4, 7
        # the affected DOF indices are (3n-1): 8, 11, 20
        F_element[8, 0] = (-bending_moment / physical_coords['P3'][0]) / 6  # Node 3
        F_element[20, 0] = 4 * (-bending_moment / physical_coords['P7'][0]) / 6  # Node 7
        F_element[11, 0] = (-bending_moment / physical_coords['P3'][0]) / 6  # Node 4

    return F_element

def apply_axial_force(N: float):

    # Apply a bending moment at node 1
    axial_force = N  # kN
    axial_force *= 1e3  # Convert to Nmm for consistency with other units
    _F = axial_force  # Convert to force per unit length

    _R = physical_coords['P2'][0]
    _r = physical_coords['P1'][0]
    _A = (_R ** 2 - _r ** 2) * np.pi  # Cross-sectional area of the cylinder

    # The bending moment is applied as a force at the node
    # For an 8-node element, we apply it to the first node (node 1)
    F_element = np.zeros((ND * NNODE, 1))

    if NNODE == 3:
        raise ValueError('Not yet implemented for 3-node elements')

    elif NNODE == 4:
        # the force is applied in the z-direction over the nodes of the top edge.
        # the affected nodes are (n): 3, 4, 7
        # the affected DOF indices are (3n-1): 8, 11, 20
        F_element[8, 0] = _F / 2  # Node 3
        F_element[11, 0] = _F / 2  # Node 4

    else:

        # the force is applied in the z-direction over the nodes of the top edge.
        # the affected nodes are (n): 3, 4, 7
        # the affected DOF indices are (3n-1): 8, 11, 20
        F_element[8, 0] = _F / 6  # Node 3
        F_element[20, 0] = 4 * _F / 6  # Node 7
        F_element[11, 0] = _F / 6  # Node 4

    return F_element

def apply_torsion(T, ng=NGS[1]):
    """
    Calculates the consistent nodal force vector from a total torsion moment.

    This function takes a total moment M, calculates the equivalent constant
    shear stress that would produce this moment over the specific element edge,
    and then computes the work-equivalent nodal forces.

    Args:
        T (float): The total torsion moment to be applied to the edge in kNm.
                  It is converted to Nmm for consistency with other units.
        ng (int): The number of Gauss points for the line integral over the edge.
        geometry_data (dict): Dictionary containing the element's geometry.
                              Needs: 'physical_coords', 'gi_data'.
        edge_nodes (list): A list of the two node indices [node_a, node_b] that form
                           the loaded edge. Example: [1, 4].

    Returns:
        np.array: The element force vector (12x1 numpy array).
    """
    # --- Unpack geometry data ---

    # --- Step 1: Calculate the integral of r^2 over the edge length s ---
    # This is needed to find the equivalent shear stress from the total moment.
    # M = 2 * pi * tau * integral(r^2 ds) => tau = M / (2 * pi * integral(r^2 ds))

    if not NNODE == 4:
        raise ValueError("Unsupported number of nodes. Only 4 nodes are supported")

    T *= 1e6  # Convert to Nmm for consistency with other units

    r1, z1 = physical_coords['P1'][0], physical_coords['P1'][2]
    r2, z2 = physical_coords['P2'][0], physical_coords['P2'][2]

    L_edge = np.sqrt((r1 - r2)**2 + (z1 - z2)**2)
    det_J_edge = L_edge / 2.0

    gauss_points = gi_data[ng]['point']
    gauss_weights = gi_data[ng]['weight']

    integral_r_squared_ds = 0.0
    for i in range(len(gauss_points)):
        s_gp = gauss_points[i]
        w_gp = gauss_weights[i]

        # 1D linear shape functions for the edge
        N_node1_at_gp = 0.5 * (1 - s_gp)
        N_node2_at_gp = 0.5 * (1 + s_gp)

        # Interpolate r at the Gauss point
        r_gp = N_node1_at_gp * r1 + N_node2_at_gp * r2

        # Add the contribution to the integral: integral(f(s) ds) = integral(f(s(xi)) * detJ * d(xi))
        integral_r_squared_ds += (r_gp**2) * det_J_edge * w_gp

    # --- Step 2: Calculate the equivalent constant shear stress ---
    if np.isclose(integral_r_squared_ds, 0):
        # Avoid division by zero if the edge is on the axis of rotation
        equivalent_shear_stress = 0.0
    else:
        equivalent_shear_stress = T / (2 * np.pi * integral_r_squared_ds)

    # --- Step 3: Call the original function with the calculated stress ---
    F_element = np.zeros((ND * NNODE, 1))
    traction_vector = np.array([[0], [equivalent_shear_stress], [0]])

    for i in range(len(gauss_points)):
        s_gp = gauss_points[i]
        w_gp = gauss_weights[i]

        N_node1_at_gp = 0.5 * (1 - s_gp)
        N_node2_at_gp = 0.5 * (1 + s_gp)
        r_gp = N_node1_at_gp * r1 + N_node2_at_gp * r2

        N_matrix_at_gp = np.zeros((3, ND * NNODE))
        N_matrix_at_gp[1, ND * 2 + 1] = N_node1_at_gp
        N_matrix_at_gp[1, ND * 3 + 1] = N_node2_at_gp

        F_contribution = 2 * np.pi * r_gp * (N_matrix_at_gp.T @ traction_vector) * det_J_edge * w_gp
        F_element += F_contribution

    return F_element


def apply_shear_force(V: float):

    raise ValueError("Unsupported shear force")

    # Apply a bending moment at node 1
    shear_force = V  # kN
    shear_force *= 1e3  # Convert to Nmm for consistency with other units
    _V = shear_force  # Convert to force per unit length

    r1, z1 = physical_coords['P1'][0], physical_coords['P1'][2]
    r2, z2 = physical_coords['P2'][0], physical_coords['P2'][2]

    L_edge = np.sqrt((r1 - r2)**2 + (z1 - z2)**2)
    det_J_edge = L_edge / 2.0

    gauss_points = gi_data[ng]['point']
    gauss_weights = gi_data[ng]['weight']

    # --- Step 3: Call the original function with the calculated stress ---
    F_element = np.zeros((ND * NNODE, 1))
    traction_vector = np.array([[0], [_V / (np.pi * (r1 + r2) / 2 * L_edge)], [0]])

    for i in range(len(gauss_points)):
        s_gp = gauss_points[i]
        w_gp = gauss_weights[i]

        N_node1_at_gp = 0.5 * (1 - s_gp)
        N_node2_at_gp = 0.5 * (1 + s_gp)
        r_gp = N_node1_at_gp * r1 + N_node2_at_gp * r2

        N_matrix_at_gp = np.zeros((3, ND * NNODE))
        N_matrix_at_gp[1, ND * 1 + 1] = N_node1_at_gp
        N_matrix_at_gp[1, ND * 2 + 1] = N_node2_at_gp

        F_contribution = np.pi * r_gp * (N_matrix_at_gp.T @ traction_vector) * det_J_edge * w_gp
        F_element += F_contribution

    return F_element

print(apply_shear_force(10))


[[   0.        ]
 [   0.        ]
 [   0.        ]
 [   0.        ]
 [4983.49550693]
 [   0.        ]
 [   0.        ]
 [5016.50449307]
 [   0.        ]
 [   0.        ]
 [   0.        ]
 [   0.        ]]


In [ ]:
if NNODE == 3:
    # fixed_dofs=[2, 5, 8, ]  # Fixed nodes for 3-node element
    fixed_dofs=[2, 5, ]  # open end
elif NNODE == 4:
    fixed_dofs=[2, 5]  # open end
elif NNODE == 8:
    fixed_dofs=[2, 5, 14]  # open end
else:
    raise ValueError("Unsupported number of nodes. Only 4 or 8 nodes are supported")

K_full = K_general(harmonic_l=0, ng=NGS[1])[0]
F_full = apply_axial_force(N=1000)
K, F = apply_boundary_conditions(K_full, F_full, fixed_dofs=fixed_dofs)
u = solve_system(K, F)
print('Case: axial force')
print("Stresses are:")
sp.pprint(get_stress_vector(u, n=0))

print()

try:
    K_full = K_general(harmonic_l=1, ng=NGS[1])[0]
    F_full = apply_bending_moment(M=1000)
    K, F = apply_boundary_conditions(K_full, F_full, fixed_dofs=fixed_dofs)
    u = solve_system(K, F)
    print('Case: bending moment')
    print("Stresses are:")
    sp.pprint(get_stress_vector(u, n=1))
except Exception as e:
    print(e)

print()


### Torsion

In [58]:
if NNODE == 3:
    # fixed_dofs=[2, 5, 8, ]  # Fixed nodes for 3-node element
    fixed_dofs=[2, 5, ]  # open end
elif NNODE == 4:
    fixed_dofs=[1, 4]  # open end
elif NNODE == 8:
    fixed_dofs=[1, 4, 13]  # open end
else:
    raise ValueError("Unsupported number of nodes. Only 4 or 8 nodes are supported")

# this will use the n=0 case, K from the B_double_bar.

try:
    K_full = K_general(harmonic_l=0, ng=NGS[1])[1]
    F_full = apply_torsion(T=100)
    K, F = apply_boundary_conditions(K_full, F_full, fixed_dofs=fixed_dofs)
    u = solve_system(K, F)
    print('Case: torsion moment')
    print("Stresses are:")
    sp.pprint(get_stress_vector(u, n=0))
except Exception as e:
    print(e)

Case: torsion moment
Stresses are:

Gauss point (s, t): -0.5774, -0.5774
sigma: Matrix([[0], [0], [0], [-4.92360412769268e-5], [0.775509758907400], [0]])

Gauss point (s, t): -0.5774, 0.5774
sigma: Matrix([[0], [0], [0], [-0.000183778825149866], [0.775509758907400], [0]])

Gauss point (s, t): 0.5774, -0.5774
sigma: Matrix([[0], [0], [0], [-4.86762929415327e-5], [0.784424967703657], [0]])

Gauss point (s, t): 0.5774, 0.5774
sigma: Matrix([[0], [0], [0], [-0.000181689504226585], [0.784424967703657], [0]])
⎡          0          ⎤
⎢                     ⎥
⎢          0          ⎥
⎢                     ⎥
⎢          0          ⎥
⎢                     ⎥
⎢-0.000181689504226585⎥
⎢                     ⎥
⎢  0.784424967703657  ⎥
⎢                     ⎥
⎣          0          ⎦


### Shear

In [60]:
if NNODE == 3:
    # fixed_dofs=[2, 5, 8, ]  # Fixed nodes for 3-node element
    fixed_dofs=[2, 5, ]  # open end
elif NNODE == 4:
    fixed_dofs=[1, 4]  # open end
elif NNODE == 8:
    fixed_dofs=[1, 4, 13]  # open end
else:
    raise ValueError("Unsupported number of nodes. Only 4 or 8 nodes are supported")

# this will use the n=1 case, K from the B_double_bar.

try:
    K_full = K_general(harmonic_l=1, ng=NGS[1])[1]
    F_full = apply_shear_force(V=100)
    K, F = apply_boundary_conditions(K_full, F_full, fixed_dofs=fixed_dofs)
    u = solve_system(K, F)
    print('Case: shear force')
    print("Stresses are:")
    sp.pprint(get_stress_vector(u, n=1))
except Exception as e:
    print(e)

Case: shear force
Stresses are:

Gauss point (s, t): -0.5774, -0.5774
sigma: Matrix([[1592046370085.60], [3714774863531.96], [1592046370085.58], [-1061364246722.72], [5044247856273.91], [-0.0218628534080031]])

Gauss point (s, t): -0.5774, 0.5774
sigma: Matrix([[5942484486921.37], [13865797136143.7], [5942484486919.83], [-3961656324609.85], [5044247856273.84], [0]])

Gauss point (s, t): 0.5774, -0.5774
sigma: Matrix([[1573946919339.94], [3672542811792.07], [1573946919339.92], [-1049297946225.61], [5044247856273.89], [0.00874514136320123]])

Gauss point (s, t): 0.5774, 0.5774
sigma: Matrix([[5874926338302.81], [13708161456033.7], [5874926338301.27], [-3916617558864.15], [5044247856273.82], [0]])
⎡5874926338302.81 ⎤
⎢                 ⎥
⎢13708161456033.7 ⎥
⎢                 ⎥
⎢5874926338301.27 ⎥
⎢                 ⎥
⎢-3916617558864.15⎥
⎢                 ⎥
⎢5044247856273.82 ⎥
⎢                 ⎥
⎣        0        ⎦
